# PySpark + MySQL for Beginner 


### First follow this blog to set up MySQL locally

https://medium.com/macoclock/mysql-on-mac-getting-started-cecb65b78e

**Step One**

enable MySQL server, instruction
https://dev.mysql.com/doc/mysql-osx-excerpt/8.0/en/osx-installation-pkg.html

Download:
https://dev.mysql.com/downloads/mysql/

Check system preference, should have mysql


**Second Two** 

SQL management tool: MySQL Workbech
https://www.mysql.com/products/workbench/

download: 
https://dev.mysql.com/downloads/workbench/


### Then follow this blog to run PySpark + MySQL

In [1]:
import pandas as pd

In [2]:
red_wines = pd.read_csv("data/winequality-red.csv", sep=";")
red_wines.shape

(1599, 12)

In [3]:
red_wines.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
red_wines["is_red"] = 1

In [5]:
white_wines = pd.read_csv("data/winequality-white.csv", sep=";")
white_wines["is_red"] = 0

In [6]:
all_wines = pd.concat([red_wines, white_wines])
all_wines

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,is_red
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


In [7]:
import mysql.connector

In [8]:
db_connection = mysql.connector.connect(host = '127.0.0.1',port = 3306, user = 'root', 
                                        passwd = '***************',db = 'mysql')
#db_cursor = db_connection.cursor()
#db_cursor.execute("CREATE DATABASE TestDB;")
#db_cursor.execute("USE TestDB;")

In [9]:
db_cursor = db_connection.cursor()
db_cursor.execute("CREATE DATABASE TestDB;")
db_cursor.execute("USE TestDB;")

In [10]:
# First time create it

db_cursor.execute("CREATE TABLE Wines(fixed_acidity FLOAT, volatile_acidity FLOAT, \
                   citric_acid FLOAT, residual_sugar FLOAT, chlorides FLOAT, \
                   free_so2 FLOAT, total_so2 FLOAT, density FLOAT, pH FLOAT, \
                   sulphates FLOAT, alcohol FLOAT, quality INT, is_red INT);")

In [11]:
wine_tuples = list(all_wines.itertuples(index=False, name=None))
wine_tuples_string = ",".join(["(" + ",".join([str(w) for w in wt]) + ")" for wt in wine_tuples])

In [12]:
db_cursor.execute("INSERT INTO Wines(fixed_acidity, volatile_acidity, citric_acid,\
                   residual_sugar, chlorides, free_so2, total_so2, density, pH,\
                   sulphates, alcohol, quality, is_red) VALUES " + wine_tuples_string + ";")
db_cursor.execute("FLUSH TABLES;")

In [13]:
db_cursor

In [14]:
from pyspark.sql import SparkSession


In [15]:
spark = SparkSession.builder.config("spark.jars", "/usr/share/java/mysql-connector-java-8.0.22.jar") \
    .master("local").appName("PySpark_MySQL_test").getOrCreate()

In [16]:
wine_df = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/TestDB") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "Wines") \
    .option("user", "root").option("password", "***************").load()

In [17]:
wine_df

DataFrame[fixed_acidity: double, volatile_acidity: double, citric_acid: double, residual_sugar: double, chlorides: double, free_so2: double, total_so2: double, density: double, pH: double, sulphates: double, alcohol: double, quality: int, is_red: int]

In [18]:
from pyspark.ml.feature import VectorAssembler

train_df, test_df = wine_df.randomSplit([.8, .2], seed=12345)
predictors = ["fixed_acidity", "volatile_acidity", "citric_acid", "residual_sugar", "chlorides",
              "free_so2", "total_so2", "density", "pH", "sulphates", "alcohol"]
vec_assembler = VectorAssembler(inputCols=predictors, outputCol="features")
vec_train_df = vec_assembler.transform(train_df)
vec_train_df.select("features", "is_red").show(5)

+--------------------+------+
|            features|is_red|
+--------------------+------+
|[3.8,0.31,0.02,11...|     0|
|[3.9,0.225,0.4,4....|     0|
|[4.2,0.17,0.36,1....|     0|
|[4.2,0.215,0.23,5...|     0|
|[4.4,0.32,0.39,4....|     0|
+--------------------+------+
only showing top 5 rows



In [19]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="is_red", featuresCol="features")
lr_model = lr.fit(vec_train_df)
vec_test_df = vec_assembler.transform(test_df)
predictions = lr_model.transform(vec_test_df)

In [20]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vec_assembler, lr])
pipeline_model = pipeline.fit(train_df)
predictions = pipeline_model.transform(test_df)

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="is_red")
evaluator.evaluate(predictions)

0.9918282206996745